In [2]:
trial = 3
f"trial_{trial:02d}"

'trial_03'

In [5]:
int("000") + int(001)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-5-daa53bcb9e2a>, line 1)

In [6]:
import h5py, numpy as np

In [7]:
x = np.random.randn(3)
y = np.random.randn(4)
print(x)
print(y)

[ 0.39355985 -0.11586729  0.19599572]
[-1.69105083  1.95044657  0.91279094 -0.70167148]


In [11]:
file = h5py.File("data.h5", "w",)
file.create_dataset(name='x', data=x, compression='gzip')
file.create_dataset(name='y', data=y, compression='gzip')
file.close()
del file

In [15]:
file = h5py.File("data.h5", "r")
print(file.keys())
print(file['x'].shape)
print(file['y'].shape)
print(file['x'][:])
print(file['y'][:])
file.close()

<KeysViewHDF5 ['x', 'y']>
(3,)
(4,)
[ 0.39355985 -0.11586729  0.19599572]
[-1.69105083  1.95044657  0.91279094 -0.70167148]


In [16]:
help(file.create_dataset)

Help on method create_dataset in module h5py._hl.group:

create_dataset(name, shape=None, dtype=None, data=None, **kwds) method of h5py._hl.files.File instance
    Create a new HDF5 dataset
    
    name
        Name of the dataset (absolute or relative).  Provide None to make
        an anonymous dataset.
    shape
        Dataset shape.  Use "()" for scalar datasets.  Required if "data"
        isn't provided.
    dtype
        Numpy dtype or string.  If omitted, dtype('f') will be used.
        Required if "data" isn't provided; otherwise, overrides data
        array's dtype.
    data
        Provide data to initialize the dataset.  If used, you can omit
        shape and dtype arguments.
    
    Keyword-only arguments:
    
    chunks
        (Tuple or int) Chunk shape, or True to enable auto-chunking. Integers can
        be used for 1D shape.
    
    maxshape
        (Tuple or int) Make the dataset resizable up to this shape. Use None for
        axes you want to be unlimited.